In [35]:
from alpaca_functions import *
import stonk_pull 
import signals
import datetime

In [36]:
# This simulates holding a current bank account that can send and receive
# money via ACH transfers to and from Alpaca
# We are assuming we have 100,000 in the bank and we are sending 100,000 to our alpaca algorithm

beginning_deposit = 100000

bank_balance = 100000

send_to_alpaca_amount = 100000

withdraw_from_alpaca_amount = 0

In [37]:
#load alpaca api keys
api = stonk_pull.load()


C:\Users\costa\desktop\byte\rice\ru-hou-fin-pt-03-2020-u-c\Project3_DoH\Final\stonk_pull.py:16: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')


In [38]:
# Send money to alpaca. This function calculates a new bank balance after we send money to alpaca
# In reality this would be an ACH transfer from our bank to alpaca.
send_to_alpaca(send_to_alpaca_amount, bank_balance) 


Transfer to alpaca successful. Your bank balance is now 0 dollars



0

In [39]:
#Pull the data from alphavantage
data = stonk_pull.data_pull(api)
data.head()

,SPY Open,SPY High,SPY Low,SPY Close,SPY Adjusted Close,SPY Volume,SPY Dividend Amount,SPY Split Coefficient
date,,,,,,,,
2020-08-21,337.92,339.72,337.5500,339.48,339.48,55106628.0,0.0,1.0
2020-08-20,335.36,338.80,335.2200,338.28,338.28,42207826.0,0.0,1.0
2020-08-19,339.05,339.61,336.6200,337.23,337.23,68054244.0,0.0,1.0
2020-08-18,338.34,339.10,336.6100,338.64,338.64,38733908.0,0.0,1.0
2020-08-17,337.94,338.34,336.8517,337.91,337.91,34496002.0,0.0,1.0


In [40]:
#Clean the data
stock_data_df = stonk_pull.data_clean(data)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
2020-08-21,337.92,339.48,55106628.0
2020-08-20,335.36,338.28,42207826.0
2020-08-19,339.05,337.23,68054244.0
2020-08-18,338.34,338.64,38733908.0
2020-08-17,337.94,337.91,34496002.0


In [41]:
#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
1999-11-01,136.5000,135.5625,4006500.0
1999-11-02,135.9687,134.5937,6516900.0
1999-11-03,136.0000,135.5000,7222300.0
1999-11-04,136.7500,136.5312,7907500.0
1999-11-05,138.6250,137.8750,7431500.0


In [42]:
#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['SPY Close'].to_frame()
signals_df.head()

,SPY Close
date,
1999-11-01,135.5625
1999-11-02,134.5937
1999-11-03,135.5000
1999-11-04,136.5312
1999-11-05,137.8750


In [43]:
#This function generates the signals dataframe
signals.generate_sma2_sma13_signals(signals_df)

,SPY Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
1999-11-01,135.5625,NaN,NaN,0.0,NaN
1999-11-02,134.5937,135.07810,NaN,0.0,0.0
1999-11-03,135.5000,135.04685,NaN,0.0,0.0
1999-11-04,136.5312,136.01560,NaN,0.0,0.0
1999-11-05,137.8750,137.20310,NaN,0.0,0.0
...,...,...,...,...,...
2020-08-17,337.9100,337.37500,332.902308,1.0,0.0
2020-08-18,338.6400,338.27500,334.031538,1.0,0.0
2020-08-19,337.2300,337.93500,334.855385,1.0,0.0


In [44]:
#Send Buy and Sell orders to alpaca using the signal, 1 = buy, -1 = sell, 0 = hold

if signals_df['Buy/Sell'].iloc[-1] == 1.0:
    symbol = "SPY"
    alpaca_functions.buy(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == -1.0:
    symbol = "SPY"
    alpaca_functions.sell(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == 0.0:
    print("The signal is hold. No action taken")

print(f"Current positions are: {api.list_positions()}")

The signal is hold. No action taken
Current positions are: []


In [45]:
#On the first of the month we will send our profits from alpaca to the bank

day_of_month = (datetime.datetime.today()).strftime("%d")

if day_of_month == 1:
    transfer_profits_from_alpaca(bank_balance, beginning_deposit)
else:
    print("No profits were transferred today.")  
    


No profits were transferred today


In [ ]:
On the first of the month we will send our profits to a cryptoexchange